In [ ]:
# Function to save radar charts for strength data
def generate_strength_radar_chart(df, athlete_id, pdf):
    athlete_data = df[(df['ID'] == athlete_id) & (df['Workout Type'] == 'Strength')]

    if athlete_data.empty:
        print("No strength test data found for this athlete.")
        return

    test_types = ['Back Squat', 'Bench Press', 'Deadlift']
    sub_types = ['Starting Strength', 'Strength-Speed', 'Accelerative Strength', 'Absolute Strength']

    fig = plt.figure(figsize=(14, 12))
    gs = GridSpec(3, 2, height_ratios=[3, 3, 3], hspace=0.3, wspace=0)

    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"

    # Set up a grouped legend
    legend_elements = [
        Line2D([0], [0], color='none', label=''),
        Line2D([0], [0], color='r', lw=5, label='Recent VBT Test'),
        Line2D([0], [0], color='b', lw=5, label='Previous VBT Test'),
        Line2D([0], [0], color='none', label=''),
        Line2D([0], [0], color='none', marker='', label='Starting Strength = 1.0-1.3 m/s'),
        Line2D([0], [0], color='none', marker='', label='Strength-Speed = 0.8-0.9 m/s'),
        Line2D([0], [0], color='none', marker='', label='Accelerative Strength = 0.5-0.7 m/s'),
        Line2D([0], [0], color='none', marker='', label='Absolute Strength = <0.4 m/s')
    ]

    for i, test_type in enumerate(test_types):
        ax = fig.add_subplot(gs[i, 0], polar=True)
        ax_table = fig.add_subplot(gs[i, 1])

        test_data = athlete_data[athlete_data['Test Type'] == test_type]
        if test_data.empty:
            print(f"No data for {test_type}")
            continue

        # Identify the previous and current phases for this specific test type
        test_data_sorted = test_data.sort_values(by=['Test Number', 'Date'], ascending=[False, False])
        phase_names = test_data_sorted['Phase'].dropna().unique()[:2]  # Extract the last two valid phases

        current_phase = phase_names[0] if len(phase_names) > 0 else 'Unknown Phase'
        previous_phase = phase_names[1] if len(phase_names) > 1 else 'Unknown Phase'

        test1_vals, test2_vals = [], []
        test1_speeds, test2_speeds = [], []

        for sub_type in sub_types:
            sub_type_data = test_data[test_data['Test Sub-Type'] == sub_type]
            if sub_type_data.empty:
                test1_vals.append(None)
                test2_vals.append(None)
                test1_speeds.append(None)
                test2_speeds.append(None)
                continue

            valid_test_numbers = sub_type_data['Test Number'].sort_values(ascending=False).unique()
            most_recent_test_number = valid_test_numbers[0]
            most_recent_test = sub_type_data[sub_type_data['Test Number'] == most_recent_test_number]

            previous_test_number = valid_test_numbers[1] if len(valid_test_numbers) > 1 else None
            previous_test = sub_type_data[sub_type_data['Test Number'] == previous_test_number] if previous_test_number else None

            test2_vals.append(most_recent_test['Weight'].values[0] if not most_recent_test.empty else None)
            test2_speeds.append(most_recent_test['Speed'].values[0] if not most_recent_test.empty else None)
            test1_vals.append(previous_test['Weight'].values[0] if previous_test is not None and not previous_test.empty else None)
            test1_speeds.append(previous_test['Speed'].values[0] if previous_test is not None and not previous_test.empty else None)

        radar_test1_vals = [0 if v is None else v for v in test1_vals]
        radar_test2_vals = [0 if v is None else v for v in test2_vals]
        radar_test1_vals += [radar_test1_vals[0]]
        radar_test2_vals += [radar_test2_vals[0]]
        angles = np.linspace(0, 2 * np.pi, len(sub_types), endpoint=False).tolist()
        angles += angles[:1]

        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(sub_types, fontsize=6, ha='center')

        if any(test1_vals):
            ax.plot(angles, radar_test1_vals, linewidth=1, linestyle='dotted', color='b', label='Test 1')
            ax.fill(angles, radar_test1_vals, 'b', alpha=0.25)

        ax.plot(angles, radar_test2_vals, linewidth=2, linestyle='solid', color='r', label='Test 2')
        ax.fill(angles, radar_test2_vals, 'r', alpha=0.25)

        max_val = max(max(radar_test1_vals), max(radar_test2_vals))
        max_y_value = max(90, np.ceil(max_val / 45) * 45)
        y_ticks = np.arange(0, max_y_value + 45, 45)
        ax.set_yticks(y_ticks)
        ax.set_yticklabels([f'{int(tick)}' for tick in y_ticks], fontsize=6)
        ax.yaxis.grid(True)

        ax.set_title(test_type, size=14, fontweight='bold')

        weight_diff = [t2 - t1 if t1 is not None and t2 is not None else None for t1, t2 in zip(test1_vals, test2_vals)]
        speed_diff = [s2 - s1 if s1 is not None and s2 is not None else None for s1, s2 in zip(test1_speeds, test2_speeds)]

        table_data = []
        for sub_type, w_diff, sp_diff in zip(sub_types, weight_diff, speed_diff):
            if w_diff is None:
                table_data.append([sub_type, 'N/A', 'N/A', 'black', 'black'])
            else:
                weight_color = 'green' if w_diff > 0 else ('red' if w_diff < 0 else 'black')
                speed_color = 'green' if sp_diff > 0 else ('red' if sp_diff < 0 else 'black')
                table_data.append([sub_type, f'{w_diff:+}', f'{sp_diff:+.2f}', weight_color, speed_color])

        col_labels = [f'{previous_phase} → {current_phase}', 'Weight Gains', 'Speed Gains']
        ax_table.axis('off')
        table = ax_table.table(cellText=[[row[0], row[1], row[2]] for row in table_data],
                               colLabels=col_labels,
                               loc='center',
                               cellLoc='center',
                               colColours=[mcolors.CSS4_COLORS['lightgray']] * 3)

        table.scale(1.2, 1.5)  # **Increase cell size**
        
        for (i, j), cell in table.get_celld().items():
            if i == 0:
                cell.set_text_props(weight='bold')

        for i, row in enumerate(table_data):
            table[(i + 1, 1)].set_text_props(color=row[3])
            table[(i + 1, 2)].set_text_props(color=row[4])

    fig.suptitle("Velocity Based Training (VBT)", fontsize=20, x=0.53, y=0.95, va='center', fontweight='bold')
    plt.figlegend(handles=legend_elements, loc='lower center', ncol=2, fontsize=10)

    pdf.savefig(fig)
    plt.close(fig)

